# **Books Website Web Srapping**

### Import Libraries

In [3]:
from bs4 import BeautifulSoup
import requests
import csv

### Read content of the website page 

In [4]:
response = requests.get("https://books.toscrape.com/")

### Parse the content to HTML

In [5]:
soup = BeautifulSoup(response.text , "html.parser")

### Scrap names of categories

In [6]:
categories = [li.find('a').get_text().strip() for li in soup.find_all("ul")[1].find_all("li")][1:]

### Scrap content of each category and save it to CSV file

In [8]:
# write on CSV file
cat_num = 1
with open("BooksToScrap3.csv","w",newline="",encoding="utf-8") as file:
    writer = csv.DictWriter(file,fieldnames=["CategoryName","BookName","Price","Rate"])
    writer.writeheader()
    # Iterate on each categorey
    for cat in categories:
        cat_num +=1
        category_name = cat.lower().replace(" ","-") + "_" + str(cat_num)
        response = requests.get("https://books.toscrape.com/catalogue/category/books/"+category_name+"/index.html")
        soup = BeautifulSoup(response.text , "html.parser")
        # Scrap the first page
        for article in soup.find_all("article",attrs={"class":"product_pod"}):
            CategoryName = cat
            BookName = article.find('h3').find("a").get("title")
            Price = article.find("div",attrs={"class":"product_price"}).find("p").get_text()[2:]
            Rate = article.find('p').get("class")[1]
            writer.writerow({"CategoryName":CategoryName, "BookName":BookName, "Price":Price, "Rate":Rate })
        
        ## Check if category has more than 1 page
        if len(soup.find("form",attrs={"class":"form-horizontal"}).find_all("strong")) > 1:
            # Scrap the other pages
            for num in range(2,int(soup.find("li",attrs={"class","current"}).get_text().strip().split(" ")[3])+1):
                response = requests.get("https://books.toscrape.com/catalogue/category/books/"+category_name+"/page-"+str(num)+".html")
                soup = BeautifulSoup(response.text , "html.parser")
                for article in soup.find_all("article",attrs={"class":"product_pod"}):
                    CategoryName = cat
                    BookName = article.find('h3').find("a").get("title")
                    Price = article.find("div",attrs={"class":"product_price"}).find("p").get_text()[2:]
                    Rate = article.find('p').get("class")[1]
                    writer.writerow({"CategoryName":CategoryName, "BookName":BookName, "Price":Price, "Rate":Rate})

        